edwith의 부스트 코스 : "파이토치로 시작하는 딥러닝 기초"를 바탕으로 작성되었습니다.  
https://www.boostcourse.org/ai214

# Softmax Classificaion
- Softmax
- Cross Entropy
- Low-level Implementation
- High-level Implementation
- Training Example

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

## Softmax
### Discrete Probability Distribution
이산적인 확률분포를 말함. 확률분포에는 연속적인 확률분포 와 이산적인 확률분포가 존재함

이산적인 확률분포를 바탕으로 Neural Network 또는 ML 을 수행하게 되는데 특정한 확률이 있을때, 다른 상황이 나타날 확률 분포를 Neural network 나 기타 다른 linear weight 들을 사용해 근사해볼 수 있음  

softmax 는 부드럽게 max 값을 출력해주는 함수이다. 각 케이스에 대해서 비율에 따라 확률값이 1이 되는 값으로 나타내주는 함수임.  

$$P(class = i) = \dfrac{e^{i}}{\sum e^{i}} = \dfrac{e^{i}}{e^{i} + e^{i+1} + e^{i+2} \dots}$$

In [3]:
z = torch.FloatTensor([1, 2, 3])

In [4]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [5]:
hypothesis.sum()

tensor(1.)

## Cross Entropy
2개의 확률 분포가 주어졌을때 그 확률 분포가 얼마나 비슷한지 나타낼 수 있는 수치  

$$L = \dfrac{1}{N}\sum -y log(\hat y)$$

In [6]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [7]:
y = torch.randint(5, (3, )).long()
print(y)
print(y.shape)
print(y.unsqueeze(1).shape)

tensor([0, 2, 1])
torch.Size([3])
torch.Size([3, 1])


In [8]:
y_one_hot = torch.zeros_like(hypothesis)
print(y_one_hot)

y_one_hot.scatter_(1, y.unsqueeze(1), 1)  # inplace 연산 

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [9]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


### Cross-entropy Loss with torch.nn.functional

In [10]:
# low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [11]:
# high level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

---

In [12]:
# low level
(y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [13]:
# high level
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward>)

nll = negative log likelihood 

In [14]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

## Training with Low-level Cross Entropy Loss
cross_entropy 손실함수를 통해서 직접 최적화 즉, 훈련학습을 하는 것을 살펴봄. 
* x_train = (m, 4)  
* y_train = (m,)

m개의 4차원 형태의 벡터가 주어졌을때, 어떠한 클래스인지 예측하고 싶은 모델  
y_train 의 값은 one_hot 벡터로 나타냈을때, one_hot 벡터에서의 1이 있는 위치의 인덱스 값임.  

이산확률분포함수를 근사하려고 하기 때문에 y_train 은 LongTensor 로 변환시켜줌

In [15]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 4, 4],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [2, 2, 2, 1, 1, 1, 0, 0]

x_train = torch.FloatTensor(x_data)
y_train = torch.LongTensor(y_data)

cross_entropy 공식  
$$L = \dfrac{1}{N}\sum -y log(\hat y)$$

In [18]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(1, nb_epochs + 1):
    # H(X)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    
    # cost : cross entropy
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot = y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번 마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch : {:4d}/{} Cost :{:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch :  100/1000 Cost :0.903006
Epoch :  200/1000 Cost :0.835022
Epoch :  300/1000 Cost :0.802080
Epoch :  400/1000 Cost :0.783067
Epoch :  500/1000 Cost :0.770440
Epoch :  600/1000 Cost :0.761184
Epoch :  700/1000 Cost :0.753931
Epoch :  800/1000 Cost :0.747978
Epoch :  900/1000 Cost :0.742930
Epoch : 1000/1000 Cost :0.738546


## Training with Low-level Cross Entropy Loss(with F.cross_entropy)
`F.cross_entropy` 를 통해서 hypothesis 와 y_train을 바로 비교해줌.  
one_hot 벡터를 만들어서 `scatter`해줄 필요가 없어지므로 이전의 방식보다 편리함. 

In [19]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(1, nb_epochs + 1):
    # H(X)
    hypothesis = x_train.matmul(W) + b
    
    # cost : cross entropy
    cost = F.cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번 마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch : {:4d}/{} Cost :{:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch :  100/1000 Cost :0.707118
Epoch :  200/1000 Cost :0.607151
Epoch :  300/1000 Cost :0.535481
Epoch :  400/1000 Cost :0.473323
Epoch :  500/1000 Cost :0.417380
Epoch :  600/1000 Cost :0.383771
Epoch :  700/1000 Cost :0.361892
Epoch :  800/1000 Cost :0.342957
Epoch :  900/1000 Cost :0.326352
Epoch : 1000/1000 Cost :0.311634


## High-level with nn.Module
보통 실전에서는 다음과 같이 class 를 만드는 방식으로 사용됨.  
`SoftmaxClassifierModel` 클래스에는 linear layer 가 존재하는 이 layer 는 4개의 input을 받아서 3개의 클래스에 대한 각각의 확률값을 output 내도록 하는 역할

In [20]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)  # 4개의 input, 3개의 output
        
    def forward(self, x):
        return self.linear(x)

In [21]:
model = SoftmaxClassifierModel()

In [22]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(1, nb_epochs + 1):
    # H(X)
    prediction = model(x_train)
    
    # cost : cross entropy
    cost = F.cross_entropy(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번 마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch : {:4d}/{} Cost :{:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch :  100/1000 Cost :0.663552
Epoch :  200/1000 Cost :0.520034
Epoch :  300/1000 Cost :0.407781
Epoch :  400/1000 Cost :0.324727
Epoch :  500/1000 Cost :0.291288
Epoch :  600/1000 Cost :0.265701
Epoch :  700/1000 Cost :0.244343
Epoch :  800/1000 Cost :0.226175
Epoch :  900/1000 Cost :0.210502
Epoch : 1000/1000 Cost :0.196829


* multinary classificaion - cross entropy - softmax
* binary classificaion(logistic regression) - binary cross entropy - sigmoid